# Download SSD model from S3, compile and deploy

This is an example to compile a pretrained MXNET SSD object detection model and deploy in sagemaker.

## Introduction

In this example, we will use the SSD mobilenet object detetion model. This model can be found in the gluoncv model zoo.

We will use the SageMaker Python SDK to host this MXNET SSD model in SageMaker, and perform inference requests.

To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.



In [ ]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base
from sagemaker import get_execution_role 

In [ ]:
role = get_execution_role() 
sess = sagemaker.Session()
region = sess.boto_region_name
account = sess.boto_session.client('sts').get_caller_identity()['Account']
bucket = sess.default_bucket()
prefix = 'pretrained-mobilenet-mxnet'

Amazon SageMaker provides prebuilt Docker images that include deep learning framework libraries and other dependencies needed for training and inference. Check here for [Prebuilt Amazon SageMaker Docker Images for TensorFlow, MXNet, Chainer, and PyTorch](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html)

In [ ]:
image_url = '301217895009.dkr.ecr.us-west-2.amazonaws.com/sagemaker-neo-mxnet:1.4.1-cpu-py3'

## The Preprocess and Postprocess Script

The `entry_point.py` script provides the preprocess and postprocess methods that we need to correctly process the input image.

In [ ]:
!cat entry_point.py

## Import pretrained mobilenet model from S3

After get the pretrained model from model zoo, we should compress the `model.json` and `model.params`  and upload the tarball to a S3 bucket for the SageMaker Python SDK to compile.

You can read more about creating an `MXNetModel` object in the [SageMaker Python SDK API docs](https://sagemaker.readthedocs.io/en/stable/sagemaker.mxnet.html#mxnet-model).

In [ ]:
from sagemaker.mxnet.model import MXNetModel
from sagemaker.predictor import RealTimePredictor

mobilenet_model = MXNetModel(
    # insert model path below
    model_data='s3:/your/path/to/model',
    image=image_url,
    entry_point='entry_point.py',
    predictor_cls=RealTimePredictor,
    role=role,
    sagemaker_session=sess,
    py_version='py3',
    framework_version='1.4.1'
)

## Compile pretrained model

Deploy using Neo API to optimize the model performance. 

In [ ]:
output_path = 's3://{}/{}/test/output'.format(bucket, prefix)
compiled_mobilenet = mobilenet_model.compile(target_instance_family='ml_c5', 
                                         input_shape={'data':[1,3,512,512]},
                                         job_name='complied-pretrained-mobilenet-mxnet22',
                                         role=role,
                                         framework='mxnet',
                                         output_path=output_path)

## Deploy compiled model

Now we will create an endpoint for the model to perform inference.

In [ ]:
compiled_mobilenet.image = image_url
compiled_mobilenet.name = 'deploy-pretrained-mobilenet-mxnet-ssd-test1'
object_detector = compiled_mobilenet.deploy(initial_instance_count = 1,
                                            instance_type = 'ml.c5.xlarge',
                                            #endpoint_name = 'deploy-pretrained-mobilenet-mxnetml-c5',
                                            #update_endpoint = True
                                           )

## Inference
### Prepare input - Image pre-processing
First convert image to byte array, then invoke entry point convert byte array back to PIL.Image readable file, after that, resize to required input size and convert to ndarray for future use.

In [ ]:
import json
import numpy as np

file_name = "test.jpg"

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload) 


### Inference with endpoint

With our Endpoint deployed successfully, we can now send inference requests to it. We'll use one image as an example here.

In [ ]:
%%time
object_detector.content_type = 'image/jpeg'
response = object_detector.predict(payload)
detections = json.loads(response)

In [ ]:
print(detections)

The results are in a format that is similar to the .lst format with an addition of a confidence score for each detected object. The format of the output can be represented as `[class_index, confidence_score, xmin, ymin, xmax, ymax]`. Typically, we don't consider low-confidence predictions.

## Visualization

We have provided additional script to easily visualize the detection outputs. You can visulize the high-confidence preditions with bounding box by filtering out low-confidence detections using the script below:

In [ ]:
%matplotlib inline
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg
        from matplotlib.patches import Rectangle

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin-2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.show()

In [ ]:
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 
                     'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 
                     'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
file_name = "test.jpg"
# Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
threshold = 0.40

# Visualize the detections.
visualize_detection(file_name, detections['prediction'], object_categories, threshold)

## Delete the Endpoint
Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [ ]:
sagemaker.Session().delete_endpoint(object_detector.endpoint)